In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# cd /content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification

# Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb
# import xgboost as xgb
from sklearn.metrics import roc_auc_score
# from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
warnings.filterwarnings('ignore')
# from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,f1_score


# Load Data

In [4]:
# train=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Train_FeatureEngineering_4_without_droping_any_feature.csv")
# test=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Test_FeatureEngineering_4.csv")
# for other account
train=pd.read_csv("features/8_Train_FeatureEngineering_fill_null_with_mean.csv")
test=pd.read_csv("features/8_Test_FeatureEngineering_fill_null_with_mean.csv")

print("Shape of Training Data",train.shape)
print("Shape of Testing Data", test.shape)


Shape of Training Data (17176, 25)
Shape of Testing Data (8576, 24)


In [5]:
train.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'label', 'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'is_outlier_loc',
       'trip_distance'],
      dtype='object')

In [6]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'is_outlier_loc',
       'trip_distance'],
      dtype='object')

# Data_Preprocessing

In [7]:
# drop_columns_train=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
# drop_columns_test=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3','is_outlier_loc']

drop_columns_train=['tripid','pickup_time','drop_time','pickup_date','pickup_day','pickup_month','pickup_year']
drop_columns_test=['tripid','pickup_time','drop_time','pickup_date','pickup_day','pickup_month','pickup_year']

In [8]:
train_1=train.drop(drop_columns_train,axis=1)
test_1=test.drop(drop_columns_test,axis=1)
print("Shape of Training Data after dropping columns",train_1.shape)
print("Shape of Testing Data after dropping columns",test_1.shape)

Shape of Training Data after dropping columns (17176, 18)
Shape of Testing Data after dropping columns (8576, 17)


In [9]:
# def encodeDays(day_of_week):
#   day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
#   return day_dict[day_of_week]
# def encodeLabel(label):
#   label_dict={'correct':1,'incorrect':0}
#   return label_dict[label]
# def clean_data(data,isTrain):
#   data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
#   if(isTrain):
#     data['label']=data['label'].apply(lambda x:encodeLabel(x))
#   return data

# train_1=clean_data(train_1,True)
# test_1=clean_data(test_1,False)
# print("Shape of Training Data after cleaning ",train_1.shape)
# print("Shape of Testing Data after cleaning", test_1.shape)

In [10]:
test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                8576 non-null   float64
 1   duration                       8576 non-null   int64  
 2   meter_waiting                  8576 non-null   int64  
 3   meter_waiting_fare             8576 non-null   float64
 4   meter_waiting_till_pickup      8576 non-null   int64  
 5   pick_lat                       8576 non-null   float64
 6   pick_lon                       8576 non-null   float64
 7   drop_lat                       8576 non-null   float64
 8   drop_lon                       8576 non-null   float64
 9   fare                           8576 non-null   float64
 10  additional_fare_is_outlier     8576 non-null   int64  
 11  meter_waiting_fare_is_outlier  8576 non-null   int64  
 12  fare_is_outlier                8576 non-null   i

In [11]:
columns_titles = ['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','pick_lat','pick_lon','drop_lat','drop_lon','fare','additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_hour','pickup_day_of_week','is_outlier_loc','trip_distance','label']
train_1=train_1.reindex(columns=columns_titles)
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                17176 non-null  float64
 1   duration                       17176 non-null  float64
 2   meter_waiting                  17176 non-null  float64
 3   meter_waiting_fare             17176 non-null  float64
 4   meter_waiting_till_pickup      17176 non-null  float64
 5   pick_lat                       17176 non-null  float64
 6   pick_lon                       17176 non-null  float64
 7   drop_lat                       17176 non-null  float64
 8   drop_lon                       17176 non-null  float64
 9   fare                           17176 non-null  float64
 10  additional_fare_is_outlier     17176 non-null  int64  
 11  meter_waiting_fare_is_outlier  17176 non-null  int64  
 12  fare_is_outlier                17176 non-null 

In [12]:
categorical_var = ['additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_hour','pickup_day_of_week','is_outlier_loc'] 
categorical_var_indices = [train_1.columns.get_loc(c) for c in categorical_var if c in train_1]
print('\nCategorical Variables indices : ',categorical_var_indices)


Categorical Variables indices :  [10, 11, 12, 13, 14, 15]


In [13]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                17176 non-null  float64
 1   duration                       17176 non-null  float64
 2   meter_waiting                  17176 non-null  float64
 3   meter_waiting_fare             17176 non-null  float64
 4   meter_waiting_till_pickup      17176 non-null  float64
 5   pick_lat                       17176 non-null  float64
 6   pick_lon                       17176 non-null  float64
 7   drop_lat                       17176 non-null  float64
 8   drop_lon                       17176 non-null  float64
 9   fare                           17176 non-null  float64
 10  additional_fare_is_outlier     17176 non-null  int64  
 11  meter_waiting_fare_is_outlier  17176 non-null  int64  
 12  fare_is_outlier                17176 non-null 

In [14]:
# train_1=pd.get_dummies(train_1)
# test_1=pd.get_dummies(test_1)

# print("Shape of Training Data after One Hot Encoding",train_1.shape)
# print("Shape of Testing Data after One Hot Encoding",test_1.shape)

In [15]:
# print('Old size: %d' % len(train))
# train_1 = train_1.dropna()
# print('New size: %d' % len(train))

In [16]:
X=train_1.drop(['label'],axis=1)
y=train_1['label']

In [17]:
#split data into train and validation data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])

Number of records in training data  13740
Number of records in validation data  3436


# Classifiers

## XGBoost CLassifier

In [18]:
# model = XGBClassifier()
# model.fit(X_train, y_train)

## Linear Regression Classifier

In [19]:
# model = LinearRegression()
# model.fit(X_train,y_train)
# # y_pred=model.predict(X_test)
# # lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
# # print("RMSE for Linear Regression is ",lm_rmse)


## LGBM Classifier

In [20]:
# estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
#                         n_estimators = 20, num_leaves = 38)


# param_grid = {
#     'n_estimators': [x for x in range(24,40,2)],
#     'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
# gridsearch = GridSearchCV(estimator, param_grid)

# gridsearch.fit(X_train, y_train,
#         eval_set = [(X_test, y_test)],
#         eval_metric = ['auc', 'binary_logloss'],
#         early_stopping_rounds = 5)

In [21]:
# gbm = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None,
#                                       colsample_bytree=1.0,
#                                       importance_type='split',
#                                       learning_rate=0.125, max_depth=-1,
#                                       metric='l1', min_child_samples=20,
#                                       min_child_weight=0.001,
#                                       min_split_gain=0.0, n_estimators=20,
#                                       n_jobs=-1, num_leaves=38, objective=None,
#                                       random_state=None, reg_alpha=0.0,
#                                       reg_lambda=0.0, silent=True,
#                                       subsample=1.0, subsample_for_bin=200000,
#                                       subsample_freq=0)


# gbm.fit(X, y,
#         eval_set=[(X_test, y_test)],
#         eval_metric=['auc', 'binary_logloss'],
# early_stopping_rounds=5)

In [22]:
# ax = lgb.plot_importance(gbm, height = 0.4, 
#                          max_num_features = 50, 
#                          xlim = (0,100), ylim = (0,23), 
#                          figsize = (10,6))
# plt.show()

In [23]:
# sorted(gbm.feature_importances_,reverse=True)

In [24]:

# # The code below aims to drop  to keep the features that are included in the most important features. 
# temp = 0 
# total = sum(gbm.feature_importances_)
# for feature in sorted(gbm.feature_importances_, reverse=True):
#     temp+=feature
#     print(feature)
#     if temp/total >= 0.85:
#         print(feature,temp/total) # stop when we 
#         break

In [25]:
# #The above means let go of all variables after PAY_AMT_5
# y_pred_prob = gbm.predict_proba(test_1)[:, 1]
# # auc_roc_0 = str(roc_auc_score(y_test, y_pred_prob)) # store AUC score without dimensionality reduction
# # print('AUC without dimensionality reduction: \n' + auc_roc_0)

In [26]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,additional_fare_is_outlier,meter_waiting_fare_is_outlier,fare_is_outlier,pickup_hour,pickup_day_of_week,is_outlier_loc,trip_distance
0,10.500000,834.000000,56.000000,0.000000,64.000000,6.86252,79.8993,6.90330,79.8783,270.32,0,0,0,0,5,0,3.164501
1,10.500000,791.000000,47.000000,0.000000,134.000000,6.88589,79.8984,6.91373,79.8923,197.85,0,0,0,0,5,0,1.968540
2,10.500000,1087.000000,80.000000,0.000000,61.000000,6.90839,79.8651,6.93669,79.9146,301.64,0,0,0,1,5,0,3.917991
3,10.500000,598.000000,271.000000,15.663800,68.000000,6.92570,79.8895,6.92748,79.8971,82.30,0,0,0,2,5,0,0.535588
4,13.719651,1702.858077,629.074231,32.057666,112.466832,6.87441,79.8615,6.84478,79.9290,358.39,1,1,0,3,5,0,5.062797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,10.500000,838.000000,93.000000,5.421900,451.000000,7.29073,80.6367,7.28891,80.6557,198.26,0,0,0,22,5,0,1.308220
17172,10.500000,2151.000000,428.000000,0.000000,39.000000,6.90569,79.8516,6.95089,79.9389,581.23,0,0,0,23,5,0,6.753297
17173,10.500000,263.000000,9.000000,0.000000,110.000000,7.09210,79.9000,7.10135,79.9017,76.20,0,0,0,23,5,0,0.649655
17174,10.500000,858.000000,115.000000,0.000000,317.000000,6.94540,79.8768,6.93574,79.9010,133.31,0,0,0,23,5,0,1.788975


In [27]:
# #We can choose to drop the last 6 features from in our new model to reduce dimensionality, and thus save training time and space

# X = X_train.drop(['additional_fare','pickup_day_of_week','pickup_month','pickup_day'], axis=1)

## CatBoost

In [28]:
# !pip install catboost

In [29]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=50000, 
#                            cat_features=categorical_var_indices, 
#                            task_type="GPU",
                           devices='0:1')
model.fit(X,
          y,
          verbose=True)




Learning rate set to 0.00096
0:	learn: 0.6921132	total: 70.8ms	remaining: 58m 58s
1:	learn: 0.6910938	total: 80.2ms	remaining: 33m 25s
2:	learn: 0.6900812	total: 89.7ms	remaining: 24m 55s
3:	learn: 0.6889395	total: 98.9ms	remaining: 20m 35s
4:	learn: 0.6880058	total: 108ms	remaining: 17m 56s
5:	learn: 0.6870512	total: 116ms	remaining: 16m 10s
6:	learn: 0.6858789	total: 125ms	remaining: 14m 56s
7:	learn: 0.6848637	total: 134ms	remaining: 13m 58s
8:	learn: 0.6839343	total: 143ms	remaining: 13m 16s
9:	learn: 0.6828690	total: 152ms	remaining: 12m 40s
10:	learn: 0.6819129	total: 161ms	remaining: 12m 10s
11:	learn: 0.6809989	total: 167ms	remaining: 11m 35s
12:	learn: 0.6799374	total: 176ms	remaining: 11m 16s
13:	learn: 0.6790109	total: 185ms	remaining: 10m 59s
14:	learn: 0.6780551	total: 195ms	remaining: 10m 49s
15:	learn: 0.6770140	total: 204ms	remaining: 10m 36s
16:	learn: 0.6760708	total: 212ms	remaining: 10m 24s
17:	learn: 0.6749374	total: 222ms	remaining: 10m 16s
18:	learn: 0.6739687	to

164:	learn: 0.5457572	total: 1.61s	remaining: 8m 5s
165:	learn: 0.5451645	total: 1.62s	remaining: 8m 5s
166:	learn: 0.5444614	total: 1.63s	remaining: 8m 5s
167:	learn: 0.5436105	total: 1.63s	remaining: 8m 4s
168:	learn: 0.5428859	total: 1.64s	remaining: 8m 4s
169:	learn: 0.5422156	total: 1.65s	remaining: 8m 4s
170:	learn: 0.5415607	total: 1.66s	remaining: 8m 4s
171:	learn: 0.5409044	total: 1.67s	remaining: 8m 4s
172:	learn: 0.5401187	total: 1.68s	remaining: 8m 4s
173:	learn: 0.5395040	total: 1.69s	remaining: 8m 4s
174:	learn: 0.5388778	total: 1.7s	remaining: 8m 4s
175:	learn: 0.5382871	total: 1.71s	remaining: 8m 3s
176:	learn: 0.5375792	total: 1.72s	remaining: 8m 3s
177:	learn: 0.5368024	total: 1.73s	remaining: 8m 3s
178:	learn: 0.5360505	total: 1.74s	remaining: 8m 3s
179:	learn: 0.5352434	total: 1.74s	remaining: 8m 2s
180:	learn: 0.5345810	total: 1.75s	remaining: 8m 2s
181:	learn: 0.5338771	total: 1.76s	remaining: 8m 2s
182:	learn: 0.5331558	total: 1.77s	remaining: 8m 2s
183:	learn: 0

337:	learn: 0.4406902	total: 3.22s	remaining: 7m 53s
338:	learn: 0.4402952	total: 3.23s	remaining: 7m 53s
339:	learn: 0.4398570	total: 3.24s	remaining: 7m 53s
340:	learn: 0.4394382	total: 3.25s	remaining: 7m 53s
341:	learn: 0.4388454	total: 3.26s	remaining: 7m 53s
342:	learn: 0.4382614	total: 3.27s	remaining: 7m 53s
343:	learn: 0.4378719	total: 3.28s	remaining: 7m 53s
344:	learn: 0.4374032	total: 3.29s	remaining: 7m 53s
345:	learn: 0.4368138	total: 3.3s	remaining: 7m 53s
346:	learn: 0.4362741	total: 3.31s	remaining: 7m 53s
347:	learn: 0.4358915	total: 3.31s	remaining: 7m 53s
348:	learn: 0.4354239	total: 3.32s	remaining: 7m 53s
349:	learn: 0.4348737	total: 3.33s	remaining: 7m 53s
350:	learn: 0.4343768	total: 3.34s	remaining: 7m 53s
351:	learn: 0.4339699	total: 3.35s	remaining: 7m 52s
352:	learn: 0.4334757	total: 3.36s	remaining: 7m 52s
353:	learn: 0.4329933	total: 3.37s	remaining: 7m 52s
354:	learn: 0.4325465	total: 3.38s	remaining: 7m 52s
355:	learn: 0.4321383	total: 3.39s	remaining: 7

502:	learn: 0.3721246	total: 4.84s	remaining: 7m 56s
503:	learn: 0.3717316	total: 4.85s	remaining: 7m 56s
504:	learn: 0.3714079	total: 4.86s	remaining: 7m 56s
505:	learn: 0.3710316	total: 4.87s	remaining: 7m 56s
506:	learn: 0.3707480	total: 4.88s	remaining: 7m 56s
507:	learn: 0.3704521	total: 4.89s	remaining: 7m 56s
508:	learn: 0.3700785	total: 4.9s	remaining: 7m 56s
509:	learn: 0.3697481	total: 4.91s	remaining: 7m 56s
510:	learn: 0.3693624	total: 4.92s	remaining: 7m 56s
511:	learn: 0.3689971	total: 4.93s	remaining: 7m 56s
512:	learn: 0.3686018	total: 4.94s	remaining: 7m 56s
513:	learn: 0.3683578	total: 4.95s	remaining: 7m 56s
514:	learn: 0.3679774	total: 4.96s	remaining: 7m 56s
515:	learn: 0.3676338	total: 4.97s	remaining: 7m 56s
516:	learn: 0.3673672	total: 4.98s	remaining: 7m 56s
517:	learn: 0.3669848	total: 4.99s	remaining: 7m 56s
518:	learn: 0.3666708	total: 5s	remaining: 7m 56s
519:	learn: 0.3663801	total: 5.01s	remaining: 7m 56s
520:	learn: 0.3660530	total: 5.01s	remaining: 7m 5

674:	learn: 0.3231723	total: 6.47s	remaining: 7m 52s
675:	learn: 0.3228980	total: 6.48s	remaining: 7m 52s
676:	learn: 0.3226244	total: 6.49s	remaining: 7m 52s
677:	learn: 0.3223755	total: 6.5s	remaining: 7m 52s
678:	learn: 0.3222110	total: 6.51s	remaining: 7m 52s
679:	learn: 0.3220281	total: 6.52s	remaining: 7m 52s
680:	learn: 0.3217791	total: 6.53s	remaining: 7m 52s
681:	learn: 0.3215674	total: 6.54s	remaining: 7m 52s
682:	learn: 0.3212858	total: 6.55s	remaining: 7m 52s
683:	learn: 0.3210325	total: 6.55s	remaining: 7m 52s
684:	learn: 0.3207714	total: 6.57s	remaining: 7m 52s
685:	learn: 0.3205640	total: 6.57s	remaining: 7m 52s
686:	learn: 0.3202964	total: 6.58s	remaining: 7m 52s
687:	learn: 0.3201467	total: 6.59s	remaining: 7m 52s
688:	learn: 0.3198487	total: 6.6s	remaining: 7m 52s
689:	learn: 0.3196089	total: 6.61s	remaining: 7m 52s
690:	learn: 0.3193806	total: 6.62s	remaining: 7m 52s
691:	learn: 0.3191673	total: 6.63s	remaining: 7m 52s
692:	learn: 0.3189964	total: 6.64s	remaining: 7m

...

49307:	learn: 0.0733967	total: 8m 1s	remaining: 6.76s
49308:	learn: 0.0733959	total: 8m 1s	remaining: 6.75s
49309:	learn: 0.0733947	total: 8m 1s	remaining: 6.74s
49310:	learn: 0.0733940	total: 8m 1s	remaining: 6.73s
49311:	learn: 0.0733933	total: 8m 1s	remaining: 6.72s
49312:	learn: 0.0733916	total: 8m 1s	remaining: 6.71s
49313:	learn: 0.0733899	total: 8m 1s	remaining: 6.7s
49314:	learn: 0.0733894	total: 8m 1s	remaining: 6.69s
49315:	learn: 0.0733879	total: 8m 1s	remaining: 6.68s
49316:	learn: 0.0733869	total: 8m 1s	remaining: 6.67s
49317:	learn: 0.0733849	total: 8m 1s	remaining: 6.66s
49318:	learn: 0.0733845	total: 8m 1s	remaining: 6.65s
49319:	learn: 0.0733831	total: 8m 1s	remaining: 6.64s
49320:	learn: 0.0733825	total: 8m 1s	remaining: 6.63s
49321:	learn: 0.0733813	total: 8m 1s	remaining: 6.62s
49322:	learn: 0.0733804	total: 8m 1s	remaining: 6.61s
49323:	learn: 0.0733793	total: 8m 1s	remaining: 6.6s
49324:	learn: 0.0733776	total: 8m 1s	remaining: 6.59s
49325:	learn: 0.0733759	total:

49468:	learn: 0.0732400	total: 8m 3s	remaining: 5.18s
49469:	learn: 0.0732395	total: 8m 3s	remaining: 5.18s
49470:	learn: 0.0732391	total: 8m 3s	remaining: 5.17s
49471:	learn: 0.0732371	total: 8m 3s	remaining: 5.16s
49472:	learn: 0.0732359	total: 8m 3s	remaining: 5.15s
49473:	learn: 0.0732348	total: 8m 3s	remaining: 5.14s
49474:	learn: 0.0732344	total: 8m 3s	remaining: 5.13s
49475:	learn: 0.0732341	total: 8m 3s	remaining: 5.12s
49476:	learn: 0.0732336	total: 8m 3s	remaining: 5.11s
49477:	learn: 0.0732317	total: 8m 3s	remaining: 5.1s
49478:	learn: 0.0732311	total: 8m 3s	remaining: 5.09s
49479:	learn: 0.0732299	total: 8m 3s	remaining: 5.08s
49480:	learn: 0.0732288	total: 8m 3s	remaining: 5.07s
49481:	learn: 0.0732269	total: 8m 3s	remaining: 5.06s
49482:	learn: 0.0732264	total: 8m 3s	remaining: 5.05s
49483:	learn: 0.0732249	total: 8m 3s	remaining: 5.04s
49484:	learn: 0.0732243	total: 8m 3s	remaining: 5.03s
49485:	learn: 0.0732238	total: 8m 3s	remaining: 5.02s
49486:	learn: 0.0732234	total

49622:	learn: 0.0730924	total: 8m 4s	remaining: 3.68s
49623:	learn: 0.0730915	total: 8m 4s	remaining: 3.67s
49624:	learn: 0.0730912	total: 8m 4s	remaining: 3.66s
49625:	learn: 0.0730899	total: 8m 4s	remaining: 3.65s
49626:	learn: 0.0730885	total: 8m 4s	remaining: 3.64s
49627:	learn: 0.0730881	total: 8m 4s	remaining: 3.63s
49628:	learn: 0.0730866	total: 8m 4s	remaining: 3.62s
49629:	learn: 0.0730861	total: 8m 4s	remaining: 3.61s
49630:	learn: 0.0730854	total: 8m 4s	remaining: 3.6s
49631:	learn: 0.0730844	total: 8m 4s	remaining: 3.59s
49632:	learn: 0.0730826	total: 8m 4s	remaining: 3.58s
49633:	learn: 0.0730815	total: 8m 4s	remaining: 3.58s
49634:	learn: 0.0730811	total: 8m 4s	remaining: 3.56s
49635:	learn: 0.0730787	total: 8m 4s	remaining: 3.56s
49636:	learn: 0.0730778	total: 8m 4s	remaining: 3.55s
49637:	learn: 0.0730763	total: 8m 4s	remaining: 3.54s
49638:	learn: 0.0730751	total: 8m 4s	remaining: 3.53s
49639:	learn: 0.0730746	total: 8m 4s	remaining: 3.52s
49640:	learn: 0.0730740	total

49787:	learn: 0.0729275	total: 8m 6s	remaining: 2.07s
49788:	learn: 0.0729271	total: 8m 6s	remaining: 2.06s
49789:	learn: 0.0729260	total: 8m 6s	remaining: 2.05s
49790:	learn: 0.0729237	total: 8m 6s	remaining: 2.04s
49791:	learn: 0.0729233	total: 8m 6s	remaining: 2.03s
49792:	learn: 0.0729226	total: 8m 6s	remaining: 2.02s
49793:	learn: 0.0729216	total: 8m 6s	remaining: 2.01s
49794:	learn: 0.0729201	total: 8m 6s	remaining: 2s
49795:	learn: 0.0729193	total: 8m 6s	remaining: 1.99s
49796:	learn: 0.0729190	total: 8m 6s	remaining: 1.98s
49797:	learn: 0.0729177	total: 8m 6s	remaining: 1.97s
49798:	learn: 0.0729173	total: 8m 6s	remaining: 1.96s
49799:	learn: 0.0729168	total: 8m 6s	remaining: 1.95s
49800:	learn: 0.0729156	total: 8m 6s	remaining: 1.94s
49801:	learn: 0.0729147	total: 8m 6s	remaining: 1.93s
49802:	learn: 0.0729137	total: 8m 6s	remaining: 1.92s
49803:	learn: 0.0729129	total: 8m 6s	remaining: 1.91s
49804:	learn: 0.0729116	total: 8m 6s	remaining: 1.9s
49805:	learn: 0.0729105	total: 8

49954:	learn: 0.0727638	total: 8m 8s	remaining: 440ms
49955:	learn: 0.0727626	total: 8m 8s	remaining: 430ms
49956:	learn: 0.0727616	total: 8m 8s	remaining: 420ms
49957:	learn: 0.0727612	total: 8m 8s	remaining: 410ms
49958:	learn: 0.0727602	total: 8m 8s	remaining: 401ms
49959:	learn: 0.0727597	total: 8m 8s	remaining: 391ms
49960:	learn: 0.0727585	total: 8m 8s	remaining: 381ms
49961:	learn: 0.0727574	total: 8m 8s	remaining: 371ms
49962:	learn: 0.0727560	total: 8m 8s	remaining: 361ms
49963:	learn: 0.0727552	total: 8m 8s	remaining: 352ms
49964:	learn: 0.0727535	total: 8m 8s	remaining: 342ms
49965:	learn: 0.0727522	total: 8m 8s	remaining: 332ms
49966:	learn: 0.0727518	total: 8m 8s	remaining: 322ms
49967:	learn: 0.0727508	total: 8m 8s	remaining: 313ms
49968:	learn: 0.0727506	total: 8m 8s	remaining: 303ms
49969:	learn: 0.0727491	total: 8m 8s	remaining: 293ms
49970:	learn: 0.0727484	total: 8m 8s	remaining: 283ms
49971:	learn: 0.0727470	total: 8m 8s	remaining: 274ms
49972:	learn: 0.0727459	tota

#Test Model

In [30]:
predictions = model.predict(X_test)
predictions = [round(value) for value in predictions]
np.unique(predictions , return_counts=True) 

(array([0, 1]), array([ 276, 3160]))

In [31]:
accuracy = accuracy_score(y_test, predictions)
f1_macro = f1_score(y_test, predictions, average='macro')
f1_binary = f1_score(y_test, predictions, average='binary')
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Macro F1: %.2f%%" % (f1_macro * 100.0))
print("Binary F1: %.2f%%" % (f1_binary * 100.0))

Accuracy: 98.05%
Macro F1: 94.02%
Binary F1: 98.93%


## Results

### catboost_categorical_100000_epochs_will_null_with_mean_without_cat_var
Accuracy: 98.17%
Macro F1: 94.42%
Binary F1: 98.99%
Submission Score : 0.97833

### catboost with categorical features fill null with mean, without categorical vars 10000 epochs 
Accuracy: 97.38%
Macro F1: 92.02%
Binary F1: 98.56%
Submission_score : 0.97803

### catboost with categorical features fill null with mean, with categorical vars 1000 epochs 
Accuracy: 96.97%
Macro F1: 90.67%
Binary F1: 98.34%

### dont know
Accuracy: 96.42%
Macro F1: 88.33%
Binary F1: 98.05%

### catboost with categorical features 100000 epochs 
Accuracy: 96.16%
Macro F1: 86.48%
Binary F1: 97.92%
Submission_score : 0.97375

### catboost without dropping any feature with 400000 epochs

Accuracy: 97.41%
Macro F1: 90.87%
Binary F1: 98.60%
Submission Score : 0.97817

### catboost without dropping any feature  with 100000 epochs

<!-- Accuracy: 97.35%
Macro F1: 90.63%
Binary F1: 98.56%
Submission Score : 0.97848 -->

### catboost after normalizing to z score with 10000 epochs

Accuracy: 96.70%
Macro F1: 87.82%
Binary F1: 98.22%

### catboost without dropping any feature with 10000 epochs

Accuracy: 96.82%
Macro F1: 88.95%
Binary F1: 98.27%
Submission Score : 0.97788

# Save Predictions

In [32]:
predictions_1 = model.predict(test_1)
# predictions_1 = [round(value) for value in y_pred_1]
np.unique(predictions_1 , return_counts=True)

(array([0, 1]), array([ 370, 8206]))

In [33]:
submission_df = pd.read_csv("predications_submission.csv", 
                            index_col="tripid")
submission_df['prediction'] = predictions_1

submission_df.to_csv('predictions/3_catboost_categorical_50000_epochs_will_null_with_mean_without_cat_var.csv')

In [34]:
#  y_pred = pd.DataFrame({'prediction':predictions_1})
#  y_pred["prediction"] = y_pred["prediction"].astype(int)
#  y_pred['prediction'] = y_pred['prediction'].apply(lambda x: 0 if x < 0.5 else 1)
#  y_pred.to_csv("predictions/predications_catboost_6_features_3_100000epochs.csv")
# y_pred.to_csv("/content/drive/My Drive/UNI/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/predictions/predications_catboost_6_features_3_400000epochs.csv")